In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
%pip install ISLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.7/350.7 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.2/522.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=24d75cd31cf8d1f055929ebaf52926665dba4f836b2e92324b52b685a5c47451
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    U

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from ISLP import load_data
Default = load_data('Default')
Default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


6a

In [3]:
x = Default[['balance', 'income']]
y = Default['default']
y = y.map(lambda x: 0 if x == 'No' else 1)

In [4]:
import statsmodels.api as sm
glm = sm.GLM(y, x, family=sm.families.Binomial())
results = glm.fit()
results.params

balance    0.000408
income    -0.000126
dtype: float64

In [5]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1734.6
Date:                Sat, 04 Nov 2023   Deviance:                       3469.1
Time:                        13:25:26   Pearson chi2:                 3.87e+04
No. Iterations:                     7   Pseudo R-squ. (CS):           -0.05638
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
balance        0.0004   7.03e-05      5.797      0.000       0.000       0.001
income        -0.0001    3.7e-06    -34.025      0.000      -0.000      -0.000
==============================================================================
"""

Estimated standard errors for coefficients are 7.03e-5 and 3.7e-6 for balance and income respectively

6b

In [65]:
def boot_fn(dataset, B):
  rng = np.random.default_rng(0)
  first_b, second_b = 0, 0
  first_i, second_i = 0, 0
  rows = dataset.shape[0]
  indices = np.arange(0, rows, 1)
  # compute first and second for each chosen index
  for _ in range(B):
    # ** replace=True
    idx = rng.choice(x.index, rows, replace=True).tolist()
    inputs = x.iloc[idx]
    truths = y.iloc[idx]
    glm = sm.GLM(truths, inputs, family=sm.families.Binomial())
    results = glm.fit()
    balance = results.params['balance']
    income = results.params['income']

    first_b += balance
    second_b += balance**2
    first_i += income
    second_i += income**2

  return [np.sqrt(second_b/B - (first_b/B)**2), np.sqrt(second_i/B - (first_i/B)**2)]

6c

In [23]:
x[x.index.duplicated()]

,balance,income


In [66]:
boot_fn(x, 1000)

[7.02535521502785e-05, 4.175319245824324e-06]

The standard error estimates with the bootstrap method are 7.024e-5 and 4.175e-6 for balance and income respectively.

6d - The standard error estimates are roughly the same for both methods.